<a href="https://colab.research.google.com/github/Filippo-Tombari/PdeGraph/blob/main/Obstacle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Risolviamo numericamente l'equazione

\begin{equation}
\frac{\partial u}{\partial t} - \Delta u + \mathbf{b}\cdot\nabla u= 0
\end{equation}

dove $\mathbf{b}=[0,1]$. Al bordo consideriamo condizioni di Neumann omogenee. Usiamo

\begin{equation}
u_{0}(x,y) = (x+1)^{2} + (y+1)^{2},
\end{equation}

come profilo iniziale.

Applicando il metodo di Eulero all'indietro in tempo abbiamo

\begin{equation}
\frac{u^{n+1} - u^{n}}{\Delta t} - \Delta u^{n+1} + \mathbf{b}\nabla u^{n+1} = 0,
\end{equation}

da cui

\begin{equation}
\implies u^{n+1} - \Delta t\Delta u^{n+1} + \Delta t\mathbf{b}\nabla u^{n+1} = u^{n}.
\end{equation}


# Import the necessary packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/content/drive/MyDrive/tesi/PdeGraph")
import os
os.chdir("/content/drive/MyDrive/tesi/PdeGraph")

Mounted at /content/drive


In [2]:
import install
install.pytorchgeo()

Pytorch geometric installed.


In [5]:
install.fenics()

FEniCS installed.


In [6]:
import numpy as np
import pickle
import functional
from functional import asfield, plot, L2, buildconnectivity, create_adj
import gnns
import dolfin
import torch
import torch.nn.functional as F
from torch_geometric.loader import NeighborSampler
import torch.optim as optim

# Loading and preparation of the data

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
mesh_train = dolfin.cpp.mesh.Mesh("files/geometry.xml")
mesh_test = dolfin.cpp.mesh.Mesh("files/geometry_test.xml")
#create edge connectivity matrices
#adj_train = torch.from_numpy(create_adj(mesh_train.cells(),mesh_train.num_vertices())).float().to(device)
edge_index_train = buildconnectivity(mesh_train)
edge_index_train = torch.t(torch.from_numpy(edge_index_train.astype('int32')).long()).to(device)
edge_index_test = buildconnectivity(mesh_test)
edge_index_test = torch.t(torch.from_numpy(edge_index_test.astype('int32')).long()).to(device)
#create festures matrices
fts_train = torch.from_numpy(np.load("files/traiettoria.npy")).float().unsqueeze(dim = 2).to(device)
fts_test = torch.from_numpy(np.load("files/traiettoria_test.npy")).float().unsqueeze(dim = 2).to(device)

print("training edge connectivity matrix shape: ", edge_index_train.shape)
print("test edge connectivity matrix shape: ", edge_index_test.shape)
#print("train adjacency matrix shape: ",adj_train.shape) #batch size x nodes x nodes
print("training feature matrix shape:", fts_train.shape)
print("test feature matrix shape:", fts_test.shape)

training edge connectivity matrix shape:  torch.Size([2, 7379])
test edge connectivity matrix shape:  torch.Size([2, 28536])
training feature matrix shape: torch.Size([11, 2553, 1])
test feature matrix shape: torch.Size([11, 9698, 1])


In [ ]:
#@title Testo del titolo predefinito
#fts_train..shape
fts_train = fts_train.unsqueeze(dim=1).expand(-1,mesh_train.num_vertices(),-1,-1)
adj_train = adj_train.expand(mesh_train.num_vertices(),-1,-1)

In [ ]:
#@title Testo del titolo predefinito
#create training, validation and test set 
torch.manual_seed(0)
valid = False
if valid == False:
  train_loader = NeighborSampler(edge_index_train, node_idx=None,
                               sizes=[-1], batch_size=fts_train.size()[1])
else:
  train_size = int(fts_train.size()[0]*0.8)
  valid_size = fts_train.size()[0] - train_size
  indices = [x for x in range(fts_train.size()[0])]
  train_idx, valid_idx = train_test_split(indices,train_size = train_size)
  train_loader = NeighborSampler(edge_index_train, node_idx=torch.Tensor(train_idx).long(),
                                sizes=[8, 7, 3], batch_size=train_size)
  valid_loader = NeighborSampler(edge_index_train, node_idx=torch.Tensor(valid_idx).long(),
                                sizes=[-1], batch_size=valid_size)
  
subgraph_loader = NeighborSampler(edge_index_test, node_idx=torch.Tensor(range(fts_test.size()[0])).long(),
                                  sizes=[-1], batch_size=fts_test.size()[0])

In [ ]:
#@title Testo del titolo predefinito
for bs, id, a in train_loader:
  batch_size_train = bs
  n_id_train = id
  adjs_train = a #`adjs` holds a list of `(edge_index, e_id, size)` tuples
if valid == True:
  for bs, id, a in valid_loader:
    batch_size_valid = bs
    n_id_valid = id
    adjs_valid = a #`adjs` holds a list of `(edge_index, e_id, size)` tuples

# Model

In [9]:
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class MPGNN(MessagePassing):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.lin_phi = torch.nn.ModuleList()
        self.lin_gamma = torch.nn.ModuleList()
        self.lin_phi.append(Linear(in_channels,hidden_channels))
        self.lin_phi.append(Linear(hidden_channels,hidden_channels*2))
        self.lin_phi.append(Linear(hidden_channels*2,hidden_channels*4))

        self.lin_gamma.append(Linear(hidden_channels*4,hidden_channels*2))
        self.lin_gamma.append(Linear(hidden_channels*2,hidden_channels))
        self.lin_gamma.append(Linear(hidden_channels,in_channels))

        self.reset_parameters()

    def reset_parameters(self):
        for nn in self.lin_phi: nn.reset_parameters()
        for nn in self.lin_gamma: nn.reset_parameters()

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        for i in range(len(self.lin_phi)):
          x = self.lin_phi[i](x)
          x = F.leaky_relu(x,negative_slope=0.1)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Start propagating messages.
        out = self.propagate(edge_index, x=x, norm=norm)

        
        return out

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j
    def update(self,out):
        for  i in range(len(self.lin_gamma)-1):
            out = self.lin_gamma[i](out)
            out = F.leaky_relu(out,negative_slope=0.1)

        out = self.lin_gamma[-1](out)
        return out

# Training

In [34]:
gnn = MPGNN(1,128,1).to(device)

In [12]:
n = 0
for tensor in list(gnn.parameters()):
  n += np.prod(tensor.shape)

n

329217

In [35]:
for nn in gnn.lin_phi:
  torch.nn.init.kaiming_normal_(nn.weight, mode='fan_out', nonlinearity='leaky_relu', a = 0.1)
for nn in gnn.lin_gamma:
  torch.nn.init.kaiming_normal_(nn.weight, mode='fan_out', nonlinearity='leaky_relu', a = 0.1)


In [33]:
l2   = L2(mesh_train).float() # L2 norm for scalar functions
lv22 = lambda v: l2(v[:,:,0].to(device)).pow(2).float() #+ l2(v[:,:,1].to(device)).pow(2).float()
lv2  = lambda v: lv22(v).sqrt().float() # L2 norm for vectorial functions
def loss(output, target):
  return (l2(target - output)/(l2(target))).mean().float()

In [36]:
#gnn = torch.load('checkpoints/basic_chk.pt')
#learningrate = 1e-2
#optimizer = optim.Adam(gnn.parameters(), lr=learningrate)
optimizer = optim.LBFGS(gnn.parameters())
model_chk_path = 'checkpoints/basic_chk.pt'
mse_min = 10000
early_stopping = 0
epochs = 100 
t = 1 # current epoch
done = False
dt = 1e-1
while not done:
      train_loss = 0
      # training
      def closure():
        optimizer.zero_grad()
        # forward pass
        #integrating = torch.stack([gnn.forward(u, edge_index_train) for u in fts_train[:]], axis = 0)
        #integrating = torch.stack([gnn.inference(u, train_loader, device) for u in fts_train[:]], axis = 0)
        #train_out = fts_train[[0]] + dt*integrating.cumsum(axis = 0) #u(t1) = u(t0) + int{t0,t1}phi(t)dt
        train_out = [gnn.forward(u, edge_index_train) for u in fts_train[:]]
        train_out = torch.stack(train_out, axis = 0)
        train_loss = loss(train_out[:-1,:,0],fts_train[1:,:,0])
        # backpropagation
        train_loss.backward()
        return train_loss
      optimizer.step(closure)
      with torch.no_grad():
        #integrating = torch.stack([gnn.forward(u, edge_index_train) for u in fts_train[:]], axis = 0)
        #integrating = torch.stack([gnn.inference(u, train_loader, device) for u in fts_train[:]], axis = 0)
        #train_out = fts_train[[0]] + dt*integrating.cumsum(axis = 0) #u(t1) = u(t0) + int{t0,t1}phi(t)dt
        train_out = [gnn.forward(u, edge_index_train) for u in fts_train[:]]
        train_out = torch.stack(train_out, axis = 0)
        train_loss = loss(train_out[:-1,:,0],fts_train[1:,:,0])
      # print rollout number and MSE for training and validation set at each epoch
     
      print(f"Rollout {t:1f}: MSE_train {train_loss.item() :6.3f}" )
      if train_loss < mse_min:
        mse_min = train_loss
        rollout_train_best = torch.cat((fts_train[[0]],train_out[:-1]),0)
        torch.save(gnn, model_chk_path)
        early_stopping = 0
        print('Saving model checkpoint')
      else:
        early_stopping += 1
      #stop the training after reaching the number of epochs
      t += 1
      #import pdb; pdb.set_trace()
      if (t > epochs ): #or early_stopping == 20
        done = True

Rollout 1.000000: MSE_train  0.107
Saving model checkpoint
Rollout 2.000000: MSE_train  0.069
Saving model checkpoint
Rollout 3.000000: MSE_train  0.039
Saving model checkpoint
Rollout 4.000000: MSE_train  0.023
Saving model checkpoint
Rollout 5.000000: MSE_train  0.020
Saving model checkpoint
Rollout 6.000000: MSE_train  0.018
Saving model checkpoint
Rollout 7.000000: MSE_train  0.017
Saving model checkpoint
Rollout 8.000000: MSE_train  0.830
Rollout 9.000000: MSE_train  5.299
Rollout 10.000000: MSE_train  4.073
Rollout 11.000000: MSE_train  3.620
Rollout 12.000000: MSE_train 61.002
Rollout 13.000000: MSE_train  4.695
Rollout 14.000000: MSE_train 36.902
Rollout 15.000000: MSE_train  5.652
Rollout 16.000000: MSE_train  2.826
Rollout 17.000000: MSE_train  0.111
Rollout 18.000000: MSE_train  0.639
Rollout 19.000000: MSE_train  0.142
Rollout 20.000000: MSE_train  0.534
Rollout 21.000000: MSE_train  0.249
Rollout 22.000000: MSE_train  0.100
Rollout 23.000000: MSE_train  0.032
Rollout 24.00

KeyboardInterrupt: ignored

In [25]:
rollout_train = torch.cat((fts_train[[0]],rollout_train_best[:-1]),0)

In [ ]:
gnn = torch.load('checkpoints/basic_chk.pt')
train_loader = NeighborSampler(edge_index_train, node_idx=None,
                               sizes=[-1], batch_size=fts_train.size()[1])

In [38]:
def forecast(u0, model, edge_index, steps, dt):
  res = [u0]
  with torch.no_grad():
    for i in range(steps):
      res.append(model.forward(res[-1],edge_index))
  return torch.stack(res, axis = 0)

In [39]:
U = forecast(fts_train[0], gnn, edge_index_train, steps = 10, dt = 1e-1)

In [ ]:
#l2   = L2(mesh_test).float() # L2 norm for scalar functions
#lv22 = lambda v: l2(v[:,:,0].to(device)).pow(2).float() #+ l2(v[:,:,1].to(device)).pow(2).float()
#lv2  = lambda v: lv22(v).sqrt().float() # L2 norm for vectorial functions
def loss(output, target):
  return (l2(target - output)).mean() #/ lv2(target)).mean().float()

In [40]:
loss(U[:,:,0],fts_train[:,:,0])

tensor(0.0558, device='cuda:0')

In [27]:
# Righe di codice per salvare l'animazione in formato .gif
import imageio
import matplotlib.pyplot as plt
def savegif(drawframe, frames, name, transparency = False, remove = True):
    filenames = []
    for i in range(frames):
        # plot frame
        drawframe(i)

        # create file name and append it to a list
        filename = f'{i}.png'
        filenames.append(filename)

        # save frame
        plt.savefig(filename, transparency = transparency)
        plt.close()
    # build gif
    with imageio.get_writer(name + '.gif', mode='I') as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # Remove files
    if(remove):
        for filename in set(filenames):
            os.remove(filename)

def trajectorytogif(traj, dt, name):
    def drawframe(i):
        colorbar = plot(functional.asfunction(traj[i][:,0], mesh_train))
        plt.colorbar(colorbar, shrink = 0.75)
        plt.title("T = %.2f" % (dt*i))
        plt.axis("off")
    savegif(drawframe, frames = len(traj), name = name)

In [41]:
trajectorytogif(U, dt, name = "images/basic_example_rollout_train") # crea e salva la gif (la si trova nella cartella dei file generati, a sx del notebook)

# Nota: su Colab non si può, ma su jupyter notebook è invece possibile visualizzare poi la gif direttamente
# dentro il notebook, e.g.

# from  IPython.display import Image as show
# show("esempio.gif")

In [ ]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Obstacle.ipynb
	modified:   gnns.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	__pycache__/
	checkpoints/obstacle_chk.pt

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git config --global user.email "filo.tombari@gmail.com"
!git config --global user.name "Filippo-Tombari"

In [ ]:
!git pull 

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/Filippo-Tombari/PdeGraph
   2e1033a..197eb60  main       -> origin/main
hint: Waiting for your editor to close the file... error: unable to start editor 'editor'
Not committing merge; use 'git commit' to complete the merge.


In [ ]:
!git add .
!git commit -m "aggiunto caso base"
!git push -u origin main

[main 4490806] aggiunto caso base
Counting objects: 27, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (27/27), done.
Writing objects: 100% (27/27), 4.22 MiB | 4.05 MiB/s, done.
Total 27 (delta 8), reused 0 (delta 0)
remote: Resolving deltas: 100% (8/8), completed with 5 local objects.
To https://github.com/Filippo-Tombari/PdeGraph.git
   197eb60..4490806  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.
